In [ ]:
import kagglehub

path = kagglehub.dataset_download("xiaojiu1414/digix-global-ai-challenge")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/xiaojiu1414/digix-global-ai-challenge/versions/1


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tqdm import tqdm
import time

In [ ]:
chunkSize = 1000000
chunks = []
index = 0
num_of_chunk = 0
train_df = pd.DataFrame()


for chunk in tqdm(pd.read_csv('/root/.cache/kagglehub/datasets/xiaojiu1414/digix-global-ai-challenge/versions/1/train/train_data_ads.csv', iterator=True, sep=',', chunksize=chunkSize)):
    num_of_chunk += 1
    chunk.reset_index(drop=True, inplace=True)
    sampled_chunk = chunk.sample(frac=0.5, replace=False, random_state=123)
    train_df = pd.concat([train_df, sampled_chunk], ignore_index=True)

1it [00:04,  4.67s/it]/usr/local/lib/python3.10/dist-packages/tqdm/std.py:1181: DtypeWarning: Columns (28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
2it [00:10,  5.21s/it]/usr/local/lib/python3.10/dist-packages/tqdm/std.py:1181: DtypeWarning: Columns (28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
3it [00:14,  4.97s/it]/usr/local/lib/python3.10/dist-packages/tqdm/std.py:1181: DtypeWarning: Columns (28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
4it [00:19,  4.98s/it]/usr/local/lib/python3.10/dist-packages/tqdm/std.py:1181: DtypeWarning: Columns (29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
5it [00:25,  5.18s/it]/usr/local/lib/python3.10/dist-packages/tqdm/std.py:1181: DtypeWarning: Columns (28,29,30) have mixed types. Specify dtype option on import o

In [ ]:
train_df.shape

(3837758, 35)

In [ ]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3837758 entries, 0 to 3837757
Data columns (total 35 columns):
 #   Column                Dtype  
---  ------                -----  
 0   log_id                int64  
 1   label                 int64  
 2   user_id               int64  
 3   age                   int64  
 4   gender                int64  
 5   residence             int64  
 6   city                  int64  
 7   city_rank             int64  
 8   series_dev            int64  
 9   series_group          int64  
 10  emui_dev              int64  
 11  device_name           int64  
 12  device_size           int64  
 13  net_type              int64  
 14  task_id               int64  
 15  adv_id                int64  
 16  creat_type_cd         int64  
 17  adv_prim_id           int64  
 18  inter_type_cd         int64  
 19  slot_id               int64  
 20  site_id               int64  
 21  spread_app_id         int64  
 22  hispace_app_tags      int64  
 23  app_sec

In [ ]:
print(f" the support of ad_click_list_v001 is {train_df['ad_click_list_v001'].unique()}")
print(f" the support of ad_click_list_v002 is {train_df['ad_click_list_v002'].unique()}")
print(f" the support of ad_click_list_v003 is {train_df['ad_click_list_v003'].unique()}")
print(f" the support of ad_close_list_v001 is {train_df['ad_close_list_v001'].unique()}")
print(f" the support of ad_close_list_v002 is {train_df['ad_close_list_v002'].unique()}")
print(f" the support of ad_close_list_v003 is {train_df['ad_close_list_v003'].unique()}")

 the support of ad_click_list_v001 is ['23769^14028^33764^22798^34197' '18536^16502^29163^30795^12889' '33764'
 ... '22763^32213^32317^20150' '31798^16066^35422^22548' '29902^10914']
 the support of ad_click_list_v002 is ['1220^1454^1690^1169^1356' '2009^1573^1633^1359^1520' '1690' ...
 '1109^1535' '1026^1968' '2009^1462']
 the support of ad_click_list_v003 is ['280^106^321^309^350' '346^350^162^343^310' '293' ...
 '372^301^280^219^344' '162^280^306^198' '306^153^240']
 the support of ad_close_list_v001 is ['24107' '11711^15603^21493^29568^29226' '17929' ...
 '20005^13874^11083^10583' '11240^22589' '26515^35353^21766^10519^14483']
 the support of ad_close_list_v002 is ['1218' '1795^1535^1036^1518' '1856' ... '1743^1090^1076' '1714^1542'
 '1597^1289^1690^1669^1036']
 the support of ad_close_list_v003 is ['173' '168^312^114' '162' ... '106^206^305^250^251' '114^162^344'
 '162^121^344^312^298']


In [ ]:
train_df.drop(['ad_click_list_v001','ad_click_list_v002', 'ad_click_list_v003', 'ad_close_list_v001', 'ad_close_list_v002','ad_close_list_v003'], axis =1,inplace = True)

In [ ]:
train_df.shape

(3837758, 29)

In [ ]:
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                     df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df

In [ ]:
train_df.drop(['u_newsCatInterestsST'], axis = 1, inplace = True)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3837758 entries, 0 to 3837757
Data columns (total 28 columns):
 #   Column            Dtype  
---  ------            -----  
 0   log_id            int64  
 1   label             int64  
 2   user_id           int64  
 3   age               int64  
 4   gender            int64  
 5   residence         int64  
 6   city              int64  
 7   city_rank         int64  
 8   series_dev        int64  
 9   series_group      int64  
 10  emui_dev          int64  
 11  device_name       int64  
 12  device_size       int64  
 13  net_type          int64  
 14  task_id           int64  
 15  adv_id            int64  
 16  creat_type_cd     int64  
 17  adv_prim_id       int64  
 18  inter_type_cd     int64  
 19  slot_id           int64  
 20  site_id           int64  
 21  spread_app_id     int64  
 22  hispace_app_tags  int64  
 23  app_second_class  int64  
 24  app_score         float64
 25  pt_d              int64  
 26  u_refreshTimes

In [ ]:
import matplotlib.pyplot as plt

cate_cols = ['log_id', 'user_id', 'age', 'gender', 'residence', 'city', 'series_dev', 'series_group', 'emui_dev', 'device_name',
             'net_type', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id', 'inter_type_cd','slot_id', 'slot_id',
             'spread_app_id', 'hispace_app_tags', 'app_second_class']

fig, axes = plt.subplots(nrows=5, ncols=6, figsize=(18, 15))
axes = axes.flatten()

# Plot each categorical column
for i, col in enumerate(cate_cols):
    value_counts = train_df[col].value_counts()  # Count each unique value for the column
    value_counts.plot(kind='bar', ax=axes[i])
    axes[i].set_title(f'{col} (Unique: {value_counts.size})')  # Show number of unique values in the title
    axes[i].set_xlabel('')
    axes[i].set_ylabel('Count')
    axes[i].tick_params(axis='x', rotation=45)

for j in range(i + 1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# for each categorical column, we create a new column that holds the count of occurrences for each unique value, which is now a numeric column
cate_cols = ['log_id', 'user_id', 'age', 'gender', 'residence', 'city', 'series_dev', 'series_group', 'emui_dev', 'device_name',
             'net_type', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id', 'inter_type_cd','slot_id', 'slot_id',
             'spread_app_id', 'hispace_app_tags', 'app_second_class']
for f in tqdm(cate_cols):
    train_df[f + '_count'] = train_df[f].map(train_df[f].value_counts())
train_df = reduce_mem(train_df)

100%|██████████| 21/21 [00:02<00:00,  8.75it/s]


-- Mem. usage decreased to 461.16 Mb (67.2% reduction),time spend:0.01 min


In [ ]:
train_df.columns

Index(['log_id', 'label', 'user_id', 'age', 'gender', 'residence', 'city',
       'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name',
       'device_size', 'net_type', 'task_id', 'adv_id', 'creat_type_cd',
       'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id',
       'hispace_app_tags', 'app_second_class', 'app_score', 'pt_d',
       'u_refreshTimes', 'u_feedLifeCycle', 'log_id_count', 'user_id_count',
       'age_count', 'gender_count', 'residence_count', 'city_count',
       'series_dev_count', 'series_group_count', 'emui_dev_count',
       'device_name_count', 'net_type_count', 'task_id_count', 'adv_id_count',
       'creat_type_cd_count', 'adv_prim_id_count', 'inter_type_cd_count',
       'slot_id_count', 'spread_app_id_count', 'hispace_app_tags_count',
       'app_second_class_count'],
      dtype='object')

In [ ]:
# Nunique encoding
def group_fea(df,key,target):
    tmp = df.groupby(key, as_index=False)[target].agg({
        key+"_"+target + '_nunique': 'nunique',
    }).reset_index()
    del tmp['index']
    print("**************************{}**************************".format(target))
    return tmp

feature_key = ['user_id','age','gender']
feature_target = ['task_id','adv_id','adv_prim_id','slot_id', 'site_id', 'spread_app_id']

for key in tqdm(feature_key):
    for target in feature_target:
        tmp = group_fea(train_df,key,target)
        train_df = train_df.merge(tmp,on=key,how='left')

  0%|          | 0/3 [00:00<?, ?it/s]<ipython-input-13-a91bec4145d6>:3: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  tmp = df.groupby(key, as_index=False)[target].agg({


**************************task_id**************************


<ipython-input-13-a91bec4145d6>:3: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  tmp = df.groupby(key, as_index=False)[target].agg({


**************************adv_id**************************


<ipython-input-13-a91bec4145d6>:3: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  tmp = df.groupby(key, as_index=False)[target].agg({


...

<ipython-input-13-a91bec4145d6>:3: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  tmp = df.groupby(key, as_index=False)[target].agg({


**************************site_id**************************


<ipython-input-13-a91bec4145d6>:3: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  tmp = df.groupby(key, as_index=False)[target].agg({


**************************spread_app_id**************************


100%|██████████| 3/3 [00:20<00:00,  6.85s/it]


In [ ]:
train_df.columns

Index(['log_id', 'label', 'user_id', 'age', 'gender', 'residence', 'city',
       'city_rank', 'series_dev', 'series_group', 'emui_dev', 'device_name',
       'device_size', 'net_type', 'task_id', 'adv_id', 'creat_type_cd',
       'adv_prim_id', 'inter_type_cd', 'slot_id', 'site_id', 'spread_app_id',
       'hispace_app_tags', 'app_second_class', 'app_score', 'pt_d',
       'u_refreshTimes', 'u_feedLifeCycle', 'log_id_count', 'user_id_count',
       'age_count', 'gender_count', 'residence_count', 'city_count',
       'series_dev_count', 'series_group_count', 'emui_dev_count',
       'device_name_count', 'net_type_count', 'task_id_count', 'adv_id_count',
       'creat_type_cd_count', 'adv_prim_id_count', 'inter_type_cd_count',
       'slot_id_count', 'spread_app_id_count', 'hispace_app_tags_count',
       'app_second_class_count', 'user_id_task_id_nunique',
       'user_id_adv_id_nunique', 'user_id_adv_prim_id_nunique',
       'user_id_slot_id_nunique', 'user_id_site_id_nunique',
      

In [ ]:
# store _nunique columns as a new group_list
group_list = []
for s in train_df.columns:
    if '_nunique' in s:
        group_list.append(s)
print(group_list)

['user_id_task_id_nunique', 'user_id_adv_id_nunique', 'user_id_adv_prim_id_nunique', 'user_id_slot_id_nunique', 'user_id_site_id_nunique', 'user_id_spread_app_id_nunique', 'age_task_id_nunique', 'age_adv_id_nunique', 'age_adv_prim_id_nunique', 'age_slot_id_nunique', 'age_site_id_nunique', 'age_spread_app_id_nunique', 'gender_task_id_nunique', 'gender_adv_id_nunique', 'gender_adv_prim_id_nunique', 'gender_slot_id_nunique', 'gender_site_id_nunique', 'gender_spread_app_id_nunique']


In [ ]:
# use cross validation in encoding to avoid bias and overfitting
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=12345)
enc_list = group_list + ['net_type', 'task_id', 'adv_id','adv_prim_id', 'age', 'app_second_class', 'city','user_id',
                         'user_id_count', 'device_name', 'slot_id']
for f in tqdm(enc_list):
  # initialize to state = 0
  train_df[f + '_target_enc'] = 0
  for i, (trn_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
    trn_x = train_df[[f, 'label']].iloc[trn_idx].reset_index(drop=True) # train column 'label'
    val_x = train_df[[f]].iloc[val_idx].reset_index(drop=True) # feature columns
    enc_df = trn_x.groupby(f, as_index=False)['label'].agg({f + '_target_enc': 'mean'}) # calculate target encoding 针对每个feature key，计算在train中feature target的平均值，并将其作为encoded feature
    val_x = val_x.merge(enc_df, on=f, how='left')
    val_x[f + '_target_enc'] = val_x[f + '_target_enc'].fillna(train_df['label'].mean())
    train_df.loc[val_idx, f + '_target_enc'] = val_x[f + '_target_enc'].values

  0%|          | 0/29 [00:00<?, ?it/s]<ipython-input-16-e05c806c0d6b>:12: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  enc_df = trn_x.groupby(f, as_index=False)['label'].agg({f + '_target_enc': 'mean'}) # calculate target encoding 针对每个feature key，计算在train中feature target的平均值，并将其作为encoded feature
<ipython-input-16-e05c806c0d6b>:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.02367808 0.01261468 0.00766078 ... 0.06254051 0.01084341 0.00822368]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_df.loc[val_idx, f + '_target_enc'] = val_x[f + '_target_enc'].values
<ipython-input-16-e05c806c0d6b>:12: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
 

In [ ]:
train_df.shape

(3837758, 95)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/train_data_2022_processed.csv'

chunk_size = 2  # You can adjust the chunk size as needed

# num of chunks
num_chunks = len(train_df) // chunk_size + (1 if len(train_df) % chunk_size > 0 else 0)

# export CSV in chunks with progress bar
with open(file_path, 'w') as f:
    for i, chunk in enumerate(tqdm(range(0, len(train_df), chunk_size), total=num_chunks, desc="Exporting CSV")):
        train_df.iloc[chunk:chunk + chunk_size].to_csv(f, header=(i == 0), index=False)

Mounted at /content/drive


Exporting CSV:  99%|█████████▉| 1908236/1918879 [1:00:48<00:17, 602.26it/s]

In [ ]:
# Sample DataFrame for nunique
data = {
    'product': ['A', 'A', 'B', 'B', 'B', 'C'],
    'sales': [100, 100, 200, 250, 300, 400],
    'my_value':[1,2,1,1,1,40]
}
df = pd.DataFrame(data)

# Group by 'product' and aggregate with sum
grouped = df.groupby('product', as_index = False)['my_value'].agg({'sales': 'nunique'})
print(grouped)
del df

  product  sales
0       A      2
1       B      1
2       C      1


<ipython-input-20-60bcf6099290>:10: FutureWarning: Passing a dictionary to SeriesGroupBy.agg is deprecated and will raise in a future version of pandas. Pass a list of aggregations instead.
  grouped = df.groupby('product', as_index = False)['my_value'].agg({'sales': 'nunique'})


In [ ]:
train_df.head()

,log_id,label,user_id,age,gender,residence,city,city_rank,series_dev,series_group,...,task_id_target_enc,adv_id_target_enc,adv_prim_id_target_enc,age_target_enc,app_second_class_target_enc,city_target_enc,user_id_target_enc,user_id_count_target_enc,device_name_target_enc,slot_id_target_enc
0,219268,0,158957,5,2,13,150,2,21,4,...,0.037037,0.037037,0.013694,0.026429,0.013049,0.015445,0.008065,0.008477,0.015471,0.018517
1,424831,0,173145,6,4,33,319,3,27,2,...,0.012435,0.012435,0.011407,0.012597,0.013543,0.013804,0.000000,0.008542,0.013405,0.030955
2,57651,0,141784,3,2,20,170,4,32,6,...,0.020668,0.020668,0.027552,0.017194,0.022755,0.014087,0.000000,0.006138,0.019927,0.036339
3,1026944,0,225669,3,2,34,187,2,16,5,...,0.010000,0.010000,0.009121,0.017262,0.013146,0.016212,0.027027,0.017914,0.013569,0.019442
4,355613,0,160800,8,2,18,417,2,32,6,...,0.021254,0.021254,0.013815,0.011909,0.011549,0.015694,0.050000,0.032626,0.018483,0.008034
